<a href="https://colab.research.google.com/github/iwatsuki-yuuki/LLM_lecture/blob/main/AdvancedPrompting_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen3-1.7B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "小学生向けに説明することを想定して、大規模言語モデルを活用するテクニックであるプロンプティングについて説明してください。600文字程度でMarkdownは使用せず、説明文のみ出力してください。"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024,
        temperature=1.0,
        top_p=0.7,
        num_return_sequences=2,
    )
for sequence in generated_ids:
    print(tokenizer.decode(sequence[len(model_inputs.input_ids[0]):], skip_special_tokens=True))
    print("-----")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

プロンプティングは、大規模言語モデル（LLM）を小学生向けに説明することを目的としたテクニックです。この方法では、シンプルでわかりやすい言葉を使って、モデルに「どう説明するか」を指示します。

例えば、「お母さんとお父さんとお子さんは一緒に遊ぶ」という文を入力すると、モデルはその文を自然な日本語で説明します。このように、プロンプティングは、モデルが複雑な内容を小学生に理解しやすいようにするための方法です。

プロンプティングには、具体的な指示や、文章の構造に注意することが重要です。また、モデルはユーザーの言葉を理解し、それに基づいて回答します。そのため、説明文を明確にし、ユーザーの意図を正確に把握することが大切です。

このテクニックは、小学生向けの教材や学習支援にとても有用です。モデルが理解しやすい言葉を使って、説明をすることで、学習効果が向上します。
-----
プロンプティングは、大規模言語モデル（LLM）に特定のタスクや要求を伝えるための技術です。小学生向けに説明する場合、シンプルでわかりやすい言葉を使って説明することが重要です。

プロンプティングの基本は、「どんなことを求めているか」を明確に伝えることです。例えば、「この問題を解け」というと、モデルは問題文を理解し、答えを出します。しかし、より具体的な指示を出すことで、モデルはより正確に答えを出します。

例えば、「5＋3＝？」と聞かれた場合、モデルは答えを出すことができます。しかし、「5＋3はいくつですか？」と聞かれた場合、答えを出すだけでなく、その計算方法も説明するよう求められます。

また、プロンプティングでは、言語の流れや文章の構造を意識するのも重要です。例えば、「この文章の意味を説明してください」と聞かれた場合、モデルは文章の内容を理解し、説明をします。

小学生向けに、プロンプティングは「どんなことを聞いているか」を明確に伝えることで、より良い答えを提供できるようにします。このようにして、言語モデルは、より自然で理解しやすい答えを出すことができます。
-----


In [ ]:
judge_prompt = """<評価対象の文章>がプロンティングについて、小学生向けにどれほど適切に説明しているか、100点満点で評価してください。
評価に際しては以下の基準を参考にしてください。
- プロンプティングとはAIへの指示の出し方であることを説明していること
- 説明の最初から小学生に語りかけるような優しい口調で説明していること
- 小学生に身近な具体例を交えて、詳しく指示出しを行うほどAIの応答が良くなることを説明していること

<評価対象の文章>
{content}
</評価対象の文章>

出力は以下のフォーマットに従ってください。
<評価の理由>
評価の理由を200文字程度で記載してください
</評価の理由>

<点数>
点数を100点満点で記載してください
</点数>"""

content = """プロンプティングは、大規模言語モデル（LLM）に指示を伝えるための方法であり、小学生向けに説明するときには分かりやすく、親しみやすい言葉で説明することが重要です。

プロンプティングとは、モデルに「何をしたいか」を具体的に伝え、モデルがそれに応じて答えを返す仕組みのことです。例えば、「この文章の続きを書け」や「100字以内でこの物語をまとめなさい」といったように、モデルに「何を求めるか」を明確に伝えます。

小学生向けには、簡単な言葉で説明し、具体的な例を挙げてわかりやすく説明するのが効果的です。また、モデルに「どんな答えをほしいか」を伝えることで、より良い回答が得られます。

プロンプティングは、単純な指示だけでなく、複雑な問題も解決するための力です。小学生でも理解しやすく、使いこなせるよう、親や先生に教えてあげると良いでしょう。"""

judge_messages = [
    {"role": "user", "content": judge_prompt.format(content=content)}
]
judge_text = tokenizer.apply_chat_template(
    judge_messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)
judge_inputs = tokenizer([judge_text], return_tensors="pt").to(model.device)
with torch.no_grad():
    judge_ids = model.generate(
        **judge_inputs,
        max_new_tokens=4096,
        do_sample=False,
    )
judge_output = tokenizer.decode(judge_ids[0][len(judge_inputs.input_ids[0]):], skip_special_tokens=True)
print(judge_output)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


<評価の理由>
文章はプロンプティングの意味を分かりやすく説明し、小学生向けに親しみやすい言葉で説明しています。具体的な例を挙げて説明し、親や先生に教えてあげると良いと述べています。内容は適切で、説明が丁寧です。

<点数>
90


In [ ]:
judge_prompt_one_shot = """<評価対象の文章>がプロンティングについて、小学生向けにどれほど適切に説明しているか、100点満点で評価してください。
評価に際しては以下の加点基準を参考にしてください。
- プロンプティングとはAIへの指示の出し方であることを説明していること
- 説明の最初から小学生に語りかけるような優しい口調で説明していること
- 小学生に身近な具体例を交えて、詳しく指示出しを行うほどAIの応答が良くなることを説明していること

出力は以下のフォーマットに従ってください。
<評価の理由>
評価の理由を200文字程度で記載してください
</評価の理由>

<点数>
点数を100点満点で記載してください
</点数>

評価の例
<評価対象の文章>
プロンプティングは、大規模言語モデルを小学生向けに説明するためのテクニックです。この方法では、モデルに「どう説明するか」を具体的に指示することで、複雑な概念を簡単に理解できるようにします。

例えば、「お母さんとお父さん」をテーマにした説明を求めるとき、モデルは「お母さんとお父さんとは、どう違うのか？」と聞かれることがあります。このとき、モデルは具体的な例やイメージを用いて、お母さんとお父さんを比較し、違いを分かりやすく説明します。

プロンプティングでは、説明の内容を「どう説明するか」に焦点を当てて指示します。このようにすることで、モデルは適切な言葉や表現を使って、小学生にも理解しやすい説明を提供できます。

また、プロンプティングは、モデルの理解力や表現力にも影響を与えます。具体的な指示をすることで、モデルはより良い説明を提供できるようになります。したがって、プロンプティングは、小学生向けに言語モデルを活用する上で非常に重要なテクニックです。
</評価対象の文章>

<評価の理由>
「プロンプティングは、大規模言語モデルを小学生向けに説明するためのテクニック」は事実として誤り。
説明の最初から語りかけているような優しい口調ではない。
具体例も不十分で、プロンプティングの効果についても浅い説明。
</評価の理由>

<点数>
50
</点数>

<評価対象の文章>
{content}
</評価対象の文章>
"""

content = """プロンプティングは、大規模言語モデル（LLM）に指示を伝えるための方法であり、小学生向けに説明するときには分かりやすく、親しみやすい言葉で説明することが重要です。

プロンプティングとは、モデルに「何をしたいか」を具体的に伝え、モデルがそれに応じて答えを返す仕組みのことです。例えば、「この文章の続きを書け」や「100字以内でこの物語をまとめなさい」といったように、モデルに「何を求めるか」を明確に伝えます。

小学生向けには、簡単な言葉で説明し、具体的な例を挙げてわかりやすく説明するのが効果的です。また、モデルに「どんな答えをほしいか」を伝えることで、より良い回答が得られます。

プロンプティングは、単純な指示だけでなく、複雑な問題も解決するための力です。小学生でも理解しやすく、使いこなせるよう、親や先生に教えてあげると良いでしょう。"""

judge_messages = [
    {"role": "user", "content": judge_prompt_one_shot.format(content=content)}
]
judge_text = tokenizer.apply_chat_template(
    judge_messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)
judge_inputs = tokenizer([judge_text], return_tensors="pt").to(model.device)
with torch.no_grad():
    judge_ids = model.generate(
        **judge_inputs,
        max_new_tokens=4096,
        do_sample=False,
    )
judge_output = tokenizer.decode(judge_ids[0][len(judge_inputs.input_ids[0]):], skip_special_tokens=True)
print(judge_output)

<評価の理由>
プロンプティングとはAIに指示を伝える方法であり、小学生向けには分かりやすく具体的な言葉で説明している。説明の内容が親しみやすいと感じられる。具体例も少なく、効果についても少し浅い説明だが、基本的な説明はされている。
</評価の理由>

<点数>
70
</点数>


In [ ]:
print(judge_output)

<評価の理由>
プロンプティングとはAIに指示を伝える方法であり、小学生向けには分かりやすく具体的な言葉で説明している。説明の内容が親しみやすいと感じられる。具体例も少なく、効果についても少し浅い説明だが、基本的な説明はされている。
</評価の理由>

<点数>
70
</点数>


LLMの出力を効果的に抜き出したい時はstructured outputを活用するのがおすすめです。

https://platform.openai.com/docs/guides/structured-outputs

In [ ]:
import re
def parse_score(output: str) -> int:
    pattern = r"<点数>\s*(\d{1,3})\s*"
    match = re.search(pattern, output)
    if match:
        return int(match.group(1))
    else:
        return -1
score = parse_score(judge_output)
print(f"Score: {score}")

Score: 70


In [ ]:
# Best-of-N
prompt = """小学生向けに説明することを想定して、大規模言語モデルを活用するテクニックであるプロンプティングについて説明してください。600文字程度でMarkdownは使用せず、説明文のみ出力してください。説明文を作成する際には以下の点を考慮してください。
- プロンプティングとはAIへの指示の出し方であることを説明していること
- 説明の最初から小学生に語りかけるような優しい口調で説明していること
- 小学生に身近な具体例を交えて、詳しく指示出しを行うほどAIの応答が良くなることを説明していること"""
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
with torch.no_grad():
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024,
        temperature=1.0,
        top_p=0.7,
        num_return_sequences=4,
    )

max_score = -1
best_output = ""
for sequence in generated_ids:
    candidate = tokenizer.decode(sequence[len(model_inputs.input_ids[0]):], skip_special_tokens=True)

    judge_messages = [
        {"role": "user", "content": judge_prompt_one_shot.format(content=candidate)}
    ]
    judge_text = tokenizer.apply_chat_template(
        judge_messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False,
    )
    judge_inputs = tokenizer([judge_text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        judge_ids = model.generate(
            **judge_inputs,
            max_new_tokens=4096,
            do_sample=False,
        )
    judge_output = tokenizer.decode(judge_ids[0][len(judge_inputs.input_ids[0]):], skip_special_tokens=True)
    score = parse_score(judge_output)
    if score == -1:
        print("Failed to parse score.")
        print(candidate)
    if score > max_score:
        max_score = score
        best_output = candidate
        print(f"New best score: {max_score}")
        print(best_output)

New best score: 50
おもちゃや遊具を一緒に遊ぶように、AIも一緒に考えてくれるよ！  
プロンプティングとは、AIに「どうしたらいいか」を伝える方法のこと。  
例えば、「お母さん、お手伝いをしましょう」というように、自分がどうしたいかを伝えるのと同じように、AIに「どうしたら答えがほしいか」を伝えるのをプロンプティングと呼ぶんだ。

小学生向けに考えると、お手伝いをしようというように、AIに「答えをほしい」という気持ちを伝えることが大切。  
例えば、「お母さん、お手伝いをしましょう」というように、自分の気持ちを伝えると、AIもそれに応じて答えをくれるよ。

具体的には、「お母さん、お手伝いをしましょう」というように、自分の気持ちを伝えると、AIもそれに応じて答えをくれるよ。  
このように、自分の気持ちを伝えることで、AIもしっかり考えて答えをくれるから、お手伝いがうまくいくよ！
New best score: 80
プロンプティングは、AIに「どうすればいいか」を教えてあげる仕組みです。簡単に言うと、AIに「何をしたいか」を具体的に説明してあげることです。

例えば、「猫の写真を描いてほしい」というと、AIはただの写真を描くだけですが、プロンプティングでは「猫が白い、大きな目をしている」というように、具体的な情報を伝えれば、AIはそれに合わせて良い写真を作ってくれます。

小学生向けに言うと、例えば「猫が白い、大きな目をしている」というように、簡単にでも伝えれば、AIはそれに従って良い写真を描いてくれます。

プロンプティングは、AIが正しい答えを出すために、どんな情報を伝えればいいかを教えてあげる仕組みです。だから、具体的で分かりやすい指示を伝えることが大切です。そうすることで、AIの応答も良くなりますよ。
New best score: 85
プロンプティングは、AIに「どうすればいいか」を伝える方法です。例えば、「おうちの犬が吠えることがあります。どうしたら吠えなくなるか」って聞かれたとき、AIはその問いに答えます。

小学生向けに考えると、例えば「おうちの猫がおにぎりを食べたいとおもうとき、どうしたらいいの？」と聞かれた場合、AIは「おにぎりを手にとればいい」と答えます。

プロンプティングは、AIが思い思いに答えを出すのではなく、私

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# import torch

# model_name = "Qwen/Qwen3-4B"

# # load the tokenizer and the model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )

# judge_prompt_one_shot = """<評価対象の文章>がプロンティングについて、小学生向けにどれほど適切に説明しているか、100点満点で評価してください。
# 評価に際しては以下の加点基準を参考にしてください。
# - プロンプティングとはAIへの指示の出し方であることを説明していること
# - 説明の最初から小学生に語りかけるような優しい口調で説明していること
# - 小学生に身近な具体例を交えて、詳しく指示出しを行うほどAIの応答が良くなることを説明していること

# 出力は以下のフォーマットに従ってください。
# <評価の理由>
# 評価の理由を200文字程度で記載してください
# </評価の理由>

# <点数>
# 点数を100点満点で記載してください
# </点数>

# 評価の例
# <評価対象の文章>
# プロンプティングは、大規模言語モデルを小学生向けに説明するためのテクニックです。この方法では、モデルに「どう説明するか」を具体的に指示することで、複雑な概念を簡単に理解できるようにします。

# 例えば、「お母さんとお父さん」をテーマにした説明を求めるとき、モデルは「お母さんとお父さんとは、どう違うのか？」と聞かれることがあります。このとき、モデルは具体的な例やイメージを用いて、お母さんとお父さんを比較し、違いを分かりやすく説明します。

# プロンプティングでは、説明の内容を「どう説明するか」に焦点を当てて指示します。このようにすることで、モデルは適切な言葉や表現を使って、小学生にも理解しやすい説明を提供できます。

# また、プロンプティングは、モデルの理解力や表現力にも影響を与えます。具体的な指示をすることで、モデルはより良い説明を提供できるようになります。したがって、プロンプティングは、小学生向けに言語モデルを活用する上で非常に重要なテクニックです。
# </評価対象の文章>

# <評価の理由>
# 「プロンプティングは、大規模言語モデルを小学生向けに説明するためのテクニック」は事実として誤り。
# 説明の最初から語りかけているような優しい口調ではない。
# 具体例も不十分で、プロンプティングの効果についても浅い説明。
# </評価の理由>

# <点数>
# 50
# </点数>

# <評価対象の文章>
# {content}
# </評価対象の文章>
# """

# import re
# def parse_score(output: str) -> int:
#     pattern = r"<点数>\s*(\d{1,3})\s*"
#     match = re.search(pattern, output)
#     if match:
#         return int(match.group(1))
#     else:
#         return -1

# # Best-of-N
# prompt = """小学生向けに説明することを想定して、大規模言語モデルを活用するテクニックであるプロンプティングについて説明してください。600文字程度でMarkdownは使用せず、説明文のみ出力してください。説明文を作成する際には以下の点を考慮してください。
# - プロンプティングとはAIへの指示の出し方であることを説明していること
# - 説明の最初から小学生に語りかけるような優しい口調で説明していること
# - 小学生に身近な具体例を交えて、詳しく指示出しを行うほどAIの応答が良くなることを説明していること"""
# messages = [
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True,
#     enable_thinking=False,
# )
# model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# # conduct text completion
# with torch.no_grad():
#     generated_ids = model.generate(
#         **model_inputs,
#         max_new_tokens=1024,
#         temperature=1.0,
#         top_p=0.7,
#         num_return_sequences=8,
#     )

# max_score = -1
# best_output = ""
# for sequence in generated_ids:
#     candidate = tokenizer.decode(sequence[len(model_inputs.input_ids[0]):], skip_special_tokens=True)

#     judge_messages = [
#         {"role": "user", "content": judge_prompt_one_shot.format(content=candidate)}
#     ]
#     judge_text = tokenizer.apply_chat_template(
#         judge_messages,
#         tokenize=False,
#         add_generation_prompt=True,
#         enable_thinking=False,
#     )
#     judge_inputs = tokenizer([judge_text], return_tensors="pt").to(model.device)
#     with torch.no_grad():
#         judge_ids = model.generate(
#             **judge_inputs,
#             max_new_tokens=4096,
#             do_sample=False,
#         )
#     judge_output = tokenizer.decode(judge_ids[0][len(judge_inputs.input_ids[0]):], skip_special_tokens=True)
#     score = parse_score(judge_output)
#     print(score)
#     if score == -1:
#         print("Failed to parse score.")
#         print(candidate)
#     if score > max_score:
#         max_score = score
#         best_output = candidate
#         print(f"New best score: {max_score}")
#         print(best_output)